install pytorch-geometric
on mac: using pip wheel installation instructions, with python 3.9 and pytorch via conda (https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html)

loading data: 
ultimately: build on-disk dataset w/ pytorch 
atm: build dataset on the fly with synthetic dataset + data loader

In [103]:
import numpy as np
# coords used in segmentation paper (polar is 2d, how?)
#relative_polar = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/relative_coords_polar_3.npy')
#relative_coord = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/relative_coords_3.npy')
#edge_index_seg = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/edge_index_3.npy')

In [100]:
import torch
from torch_geometric.data import Data
import numpy as np
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import Cartesian, Polar


In [132]:
def get_fake_edge_index(level=1):
    # TODO define this for multiple levels for the real data
    edge_index = torch.tensor([[0, 1],
                            [1, 0],
                            [1, 2],
                            [2, 1],[0,0],[1,1],[2,2]], dtype=torch.long)
    # get relative cartensian / polar coords as edge_attrs to use a pseudo coordinates later on
    position = torch.tensor([[0,0],[1,1],[2,0]], dtype=torch.float)
    data = Data(edge_index=edge_index.t().contiguous(), pos=position)
    edge_attr_polar = Polar(norm=True, cat=False)(data).edge_attr  # TODO figure out how exactly want to do coord transform (should be relative difference between nodes)
    return edge_index.t().contiguous(), edge_attr_polar

def get_fake_data():
    # fake some patient data with 3 nodes and per-node labels, and 5 features
    # build data list
    data_list = []
    for i in range(100):
        x = torch.tensor(np.random.rand(3,5), dtype=torch.float)
        y = torch.tensor(np.random.choice([0,1], size=(3,1)))
        data_list.append(Data(x=x, y=y))

    loader = DataLoader(data_list, batch_size=2)

    # can put loader + model on GPU here
    # could use transforms on data loader here
    # example loading of data + custom dataset for brain data: https://github.com/Abdulah-Fawaz/Benchmarking-Surface-DL/blob/main/Segmentation_UGSCNN/GraphMethods/segmentation.py

    return loader


In [102]:
for batch in loader:
    print(batch)
    print(batch.num_graphs)

DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2
DataBatch(x=[6, 5], edge_index=[2, 14], edge_attr=[14, 2], y=[6, 1], batch=[6], ptr=[3])
2

In [208]:
import torch
from torch_geometric.nn import GMMConv

# define model
class MoNet(torch.nn.Module):
    def __init__(self, num_features, layer_sizes, dim=2, kernel_size=3, edge_index=get_fake_edge_index):
        """
        dim: dim for GMMConv, dimension of coord representation - 2 or 3
        kernel_size: number of kernels (default 3)
        layer_sizes: (list) output size of each conv layer. a final linear layer for going to 2 (binary classification) is added
        num_features: number of input features (input size)
        """
        super(MoNet, self).__init__()
        self.num_features = num_features
        assert len(layer_sizes) >= 1
        layer_sizes.insert(0, num_features)
        self.layer_sizes = layer_sizes
        conv_layers = []
        for in_size, out_size in zip(layer_sizes[:-1], layer_sizes[1:]):
            conv_layers.append(GMMConv(in_size, out_size, dim=dim, kernel_size=kernel_size))
        self.conv_layers = torch.nn.Sequential(*conv_layers)
        self.fc = torch.nn.Linear(self.layer_sizes[-1], 2)
        self.activation_function = torch.nn.ReLU()
        self.edge_indices = [edge_index(level=1)[0]]
        self.edge_attrs = [edge_index(level=1)[1]]

        # initialisation - TODO figure out what we want
        # in segmentation benchmark they don't do any -- use default

    def forward(self, data):
        x = data
        for cl in self.conv_layers:
            x = cl(x, self.edge_indices[0], self.edge_attrs[0])
            x = self.activation_function(x)
        # add final linear layer
        x = self.activation_function(self.fc(x))
        return torch.nn.LogSoftmax(dim=1)(x)


model = MoNet(num_features=5, layer_sizes=[16,10], dim=2, edge_index=get_fake_edge_index)
# TODO try and fit model
# think about initialisatiomn
# coord rep


In [213]:
import os
from copy import deepcopy
import logging

network_parameters = {
    'network_type': 'MoNet',
    'model_parameters': {
        'layer_sizes': [16,10],
        'dim': 2, # pseudo-coord dim
        'num_features': 5 # number of input features
    },
    "max_patience": 10,
    "num_epochs": 100,
    'lr': 1e-4
}

class Experiment:
    def __init__(self, network_parameters):
        self.network_parameters = network_parameters
        # TODO save parameters

        self.model = None # loaded by self.load_model()
        self.log = logging.getLogger(__name__)

    def from_folder(self, path):
        pass

    def load_model(self, checkpoint_path=None):
        """
        build model and optionally load weights from checkpoint
        """
        # build model using network_parameters
        network_type = self.network_parameters['network_type']
        if network_type == 'MoNet':
            self.model = MoNet(**self.network_parameters['model_parameters'], edge_index=get_fake_edge_index)
        else:
            raise(NotImplementedError, network_type)

        # load model weights

        #torch.save(model.state_dict(), PATH)

        if checkpoint_path is not None and os.path.isdir(checkpoint_path):
            # checkpoint contains both model architecture + weights
            self.log.info("Loading model weights from checkpoint")
            self.model.load_state_dict(torch.load(checkpoint_path))
            self.model.eval()

    def train(self, save_path=None):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
        # set up model
        if self.model is None:
            self.load_model()
        self.model.to(device)

        # get data
        train_data_loader = get_fake_data()
        val_data_loader = get_fake_data()
        # get edge_

        # set up training loop
        optimiser = torch.optim.Adam(self.model.parameters(),lr=self.network_parameters['lr'])
    
        validation_losses = []
        train_losses = []
    
        best_loss = 100000
        patience = 0
        for epoch in range(self.network_parameters['num_epochs']):
            running_losses = []
            
            for i, data in enumerate(train_data_loader):  
                data = data.to(device)
                self.model.train()     
                optimiser.zero_grad()
                estimates = self.model(data.x)
                labels = data.y.squeeze()
                loss = torch.nn.NLLLoss()(estimates, labels)
                loss.backward()
                optimiser.step()
                running_losses.append(loss.item())
          
            print('Epoch {} :: Train loss {:.3f}'.format(epoch,np.mean(running_losses)))
            train_losses.append(np.mean(running_losses))
        
            if epoch%1 ==0:
                with torch.no_grad():
                    running_losses  = []
                    for i, data in enumerate(val_data_loader):
                        data = data.to(device)
                        estimates = self.model(data.x)
                        labels = data.y.squeeze()
                        loss = torch.nn.NLLLoss()(estimates, labels)
                        running_losses.append(loss.item())
                            
                    val_loss = np.mean(running_losses)
                    validation_losses.append(val_loss)
                    print('validation ', val_loss)
                        
                    if val_loss < best_loss:
                        best_loss = val_loss
                        if save_path is not None:
                            torch.save(self.model.state_dict(), save_path)
                            print('saved_new_best')
                        patience = 0
                    else:
                        patience+=1
                    if patience >= self.network_parameters['max_patience']:
                        break
                    
                    print('----------')
        # save final model
        #torch.save(model,'/home/lw19/Desktop/final_benchmarking_models/final_GConvNet_TopK_Native_warp_no_rot')

In [214]:
exp = Experiment(network_parameters)

In [215]:
exp.load_model()

In [216]:
exp.train()

Epoch 0 :: Train loss 0.693
validation  0.6931207931041717
----------
Epoch 1 :: Train loss 0.693
validation  0.6931080949306488
----------
Epoch 2 :: Train loss 0.693
validation  0.693099069595337
----------
Epoch 3 :: Train loss 0.693
validation  0.693090192079544
----------
Epoch 4 :: Train loss 0.693
validation  0.6930792272090912
----------
Epoch 5 :: Train loss 0.693
validation  0.6930769991874695
----------
Epoch 6 :: Train loss 0.693
validation  0.6930726075172424
----------
Epoch 7 :: Train loss 0.693
validation  0.6930650472640991
----------
Epoch 8 :: Train loss 0.692
validation  0.6930631458759308
----------
Epoch 9 :: Train loss 0.692
validation  0.6930594158172607
----------
Epoch 10 :: Train loss 0.692
validation  0.6930659806728363
----------
Epoch 11 :: Train loss 0.692
validation  0.6930728471279144
----------
Epoch 12 :: Train loss 0.692
validation  0.6930850327014924
----------
Epoch 13 :: Train loss 0.692
validation  0.6930976104736328
----------
Epoch 14 :: Train 